# Importing Libraries

In [24]:
!pip install ctransformers>=0.2.24
!pip install langchain
!pip install chromadb
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
import numpy as np
import os
import random

# Data Preprocessing

In [5]:
data=pd.read_csv("/content/A_Z_medicines_dataset_of_India.csv")

<ipython-input-5-1222a80096d5>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/A_Z_medicines_dataset_of_India.csv")


In [6]:
df = data[data['Is_discontinued'] != True]

In [7]:
df.count()
df.head()

,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN


# Data to Sentences

In [8]:
df.loc[:,'text'] = df['name'] + " priced at ₹" + df['price(₹)'].astype(str) + " by " + df['manufacturer_name'] + " is a type of " + df['type'] + ". It has " + df['short_composition1'] + "and" + df['short_composition2'].apply(lambda x: " " + x if pd.notna(x) else "") + ". It has a pack size label of " + df['pack_size_label']


<ipython-input-8-3b0247617cca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'text'] = df['name'] + " priced at ₹" + df['price(₹)'].astype(str) + " by " + df['manufacturer_name'] + " is a type of " + df['type'] + ". It has " + df['short_composition1'] + "and" + df['short_composition2'].apply(lambda x: " " + x if pd.notna(x) else "") + ". It has a pack size label of " + df['pack_size_label']


In [9]:
docs=df['text'].tolist()

In [10]:
docs = random.sample(docs, 100)

In [11]:
docs[0:10]

['Etb 90mg Tablet priced at ₹85.0 by Accilex Nutricorp is a type of allopathy. It has Etoricoxib (90mg)and. It has a pack size label of strip of 10 tablets',
 'Cadcef 200mg Tablet priced at ₹127.0 by Cadman Healthcare is a type of allopathy. It has Cefixime (200mg)and. It has a pack size label of strip of 10 tablets',
 'Coverit Plus Spray priced at ₹980.0 by Micro Labs Ltd is a type of allopathy. It has Minoxidil (5% w/v) and  Finasteride (0.1% w/v). It has a pack size label of bottle of 100 ml Solution',
 'Olip 7.5mg Tablet priced at ₹45.0 by PJ Pharmaceuticals is a type of allopathy. It has Olanzapine (7.5mg)and. It has a pack size label of strip of 10 tablets',
 'Mupirin 2% Cream priced at ₹98.0 by Aeryn Lifesciences Pvt Ltd is a type of allopathy. It has Mupirocin (2% w/w)and. It has a pack size label of tube of 5 gm Cream',
 'Mikaril 500mg Injection priced at ₹57.0 by Merril Pharma Pvt Ltd is a type of allopathy. It has Amikacin (500mg)and. It has a pack size label of vial of 2 ml

# Adding them to Vector

In [12]:
client = chromadb.Client()
collection = client.get_or_create_collection("A-Z_Medicine")

In [13]:
ids= [str(x) for x in df.index.tolist()]

In [14]:
chunk_size = 100

for i in range(0, len(docs), chunk_size):
  chunk_docs = docs[i:i+chunk_size]
  chunk_ids = ids[i:i+chunk_size]
  collection.add(
    documents=chunk_docs,
    ids=chunk_ids
)
  print(i)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:05<00:00, 15.2MiB/s]


0


In [15]:
results = collection.query(
    query_texts=["Zess 500mg"],
    n_results=10
)

In [16]:
print(results['documents'])

[['Letin AM 5mg/75mg Tablet SR priced at ₹57.5 by Insight Labz is a type of allopathy. It has Levocetirizine (5mg) and  Ambroxol (75mg). It has a pack size label of strip of 10 tablet sr', 'Gatosart 80 Tablet priced at ₹100.0 by Cmg Biotech Pvt Ltd is a type of allopathy. It has Valsartan (80mg)and. It has a pack size label of strip of 10 tablets', 'Lutiphase 300mg Tablet SR priced at ₹450.0 by Voizmed Pharma Pvt Ltd is a type of allopathy. It has Progesterone (Natural Micronized) (300mg)and. It has a pack size label of strip of 10 tablet sr', 'Lypex SB 500 mg/500 mg Injection priced at ₹236.0 by Hetero Drugs Ltd is a type of allopathy. It has Cefoperazone (500mg) and  Sulbactam (500mg). It has a pack size label of vial of 1 Injection', 'Amoxise CV 500mg/125mg Tablet priced at ₹140.0 by Carise Pharmaceuticals Pvt Ltd is a type of allopathy. It has Amoxycillin  (500mg) and   Clavulanic Acid (125mg). It has a pack size label of strip of 10 tablets', 'Lavostat 500mg Tablet priced at ₹61.0

# RAG with LLMA2

In [25]:
from langchain.llms import CTransformers
llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF", model_file="llama-2-13b-ensemble-v5.Q5_K_M.gguf", model_type="llama")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-13b-ensemble-v5.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [17]:
def text_embedding(text) -> None:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text)

In [18]:
def generate_context(query):
    vector=text_embedding(query).tolist()

    results=collection.query(
        query_embeddings=vector,
        n_results=5,
        include=["documents"]
    )

    res = "\n".join(str(item) for item in results['documents'][0])
    return res

In [20]:
query="What is the price of Letin AM 5mg/75mg tablet"
context=generate_context(query)
context

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'Letin AM 5mg/75mg Tablet SR priced at ₹57.5 by Insight Labz is a type of allopathy. It has Levocetirizine (5mg) and  Ambroxol (75mg). It has a pack size label of strip of 10 tablet sr\nOdcan 50mg Tablet priced at ₹82.0 by Prism Life Sciences Ltd is a type of allopathy. It has Fluconazole (50mg)and. It has a pack size label of strip of 4 tablets\nEtokite 90mg Tablet priced at ₹110.0 by Torance Healthcare Pvt Ltd is a type of allopathy. It has Etoricoxib (90mg)and. It has a pack size label of strip of 10 tablets\nMefezy 500mg Tablet priced at ₹27.0 by Truecure Healthcare Private Limited is a type of allopathy. It has Mefenamic Acid (500mg)and. It has a pack size label of strip of 10 tablets\nDefidrot 80mg Tablet priced at ₹69.0 by Ridhima Biocare is a type of allopathy. It has Drotaverine (80mg)and. It has a pack size label of strip of 10 tablets'

In [21]:
system_prompt="""\
You are a helpful AI assistant that can answer questions on A-z medicines. Answer based on the context provided. If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
"""

In [22]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
"""

In [33]:
from langchain import PromptTemplate,  LLMChain

template = """
              Based on the context:""" + context + """
Answer the below query:
{text}
You are a helpful AI assistant that can answer questions on A-z medicines. Answer based on the context provided. If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
           """


prompt = PromptTemplate(template=template, input_variables=["text"])




llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['text'], template="\n              Based on the context:Letin AM 5mg/75mg Tablet SR priced at ₹57.5 by Insight Labz is a type of allopathy. It has Levocetirizine (5mg) and  Ambroxol (75mg). It has a pack size label of strip of 10 tablet sr\nOdcan 50mg Tablet priced at ₹82.0 by Prism Life Sciences Ltd is a type of allopathy. It has Fluconazole (50mg)and. It has a pack size label of strip of 4 tablets\nEtokite 90mg Tablet priced at ₹110.0 by Torance Healthcare Pvt Ltd is a type of allopathy. It has Etoricoxib (90mg)and. It has a pack size label of strip of 10 tablets\nMefezy 500mg Tablet priced at ₹27.0 by Truecure Healthcare Private Limited is a type of allopathy. It has Mefenamic Acid (500mg)and. It has a pack size label of strip of 10 tablets\nDefidrot 80mg Tablet priced at ₹69.0 by Ridhima Biocare is a type of allopathy. It has Drotaverine (80mg)and. It has a pack size label of strip of 10 tablets\nAnswer the below query:\n{text}\nYou a

In [34]:
text1 = """
What is the price of Letin AM tablet
"""
output = llm_chain.run(text1)
print(output)


 The price of Letin AM 5mg/75mg Tablet SR is ₹57.5 by Insight Labz.
            
        
